## EDA: Relationship between Video Length & Number of View Actions

In this section, we will explore the relationship between the length of the videos and the number of view actions.

### Steps:
1. Extract the unique values of video names.
2. Calculate the number of reviews for each video.
3. Merge the review data with view action counts.
4. Calculate the total view actions for each video.
5. Calculate the total video length for each video.
6. Analyze the correlation between video length and view actions.

### Results:
- Display the top videos by the number of reviews.
- Display the top videos by the number of view actions.
- Show the correlation between video length and total view actions.

**Correlation between video length and total view actions: 0.7981207829817402**

#### Import Package

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

#### Read csv file

In [2]:
# Read the CSV files into DataFrames
df_user = pd.read_csv('../2024初賽用開放資料/user_data.csv')
df_exam = pd.read_csv('../2024初賽用開放資料/dp001_exam.csv')
df_prac = pd.read_csv('../2024初賽用開放資料/dp001_prac.csv')
df_review = pd.read_csv('../2024初賽用開放資料/dp001_review.csv')
df_review_plus = pd.read_csv('../2024初賽用開放資料/dp001_review_plus.csv')
df_exam2 = pd.read_csv('../2024初賽用開放資料/dp002_exam.csv')
df_math = pd.read_csv('../2024初賽用開放資料/dp003_math.csv')
df_word = pd.read_csv('../2024初賽用開放資料/dp003_word.csv')

#### COMBINE Data Frame

In [3]:
df_user_review = df_user.merge(df_review, on='user_sn', how='left')
df_combined = df_user_review.merge(df_review_plus, on='review_sn', how='left')
df_filtered = df_combined[~df_combined['review_sn'].isin(df_exam['review_sn'])]

df_filtered.to_csv('df_dp001_review.csv', index=False)


### Data Analysis


In [4]:
print('影片總共有：', df_review['video_name'].nunique(),'種')
# 計算每個 video_name 對應的 review_sn 數量
video_review_counts = df_review.groupby('video_name')['review_sn'].nunique().reset_index()
# 按照 review_sn 數量排序並顯示前幾個
top_video_reviews = video_review_counts.sort_values(by='review_sn', ascending=False).head()
display(top_video_reviews)

影片總共有： 274 種


,video_name,review_sn
74,video:5-n-08-S03,70
82,video:5-n-10-S05,67
110,video:5-n-20-S03,65
93,video:5-n-14-S03,64
54,video:4-n-15-S02,63


In [5]:
# 1. 提取 video_name 和 review_sn 的對應關係
video_review_mapping = df_review[['video_name', 'review_sn', 'video_len']].drop_duplicates()
video_review_mapping['video_len'] = video_review_mapping['video_len'].round()

# 2. 使用 review_sn 在 df_review_plus 中計算每個 review_sn 的 view_action 數量
view_action_counts = df_review_plus.groupby('review_sn')['view_action'].count().reset_index()

# 3. 合併 video_review_mapping 和 view_action_counts
video_action_counts = video_review_mapping.merge(view_action_counts, on='review_sn', how='left')

# 4. 計算每個 video_name 的總 view_action 數量
total_action_counts = video_action_counts.groupby('video_name')['view_action'].sum().reset_index()

# 5. 計算每個 video_name 的 video_len
total_video_len = df_review[['video_name', 'video_len']].drop_duplicates()

# 6. 合併 total_action_counts 和 total_video_len
video_stats = total_action_counts.merge(total_video_len, on='video_name')

# 7. 計算每個 video_name 的 review_sn 數量
review_sn_counts = video_review_mapping.groupby('video_name')['review_sn'].count().reset_index()
review_sn_counts.rename(columns={'review_sn': 'review_sn_count'}, inplace=True)

# 8. 合併 review_sn_counts 和 video_stats
video_stats = video_stats.merge(review_sn_counts, on='video_name')

# 9. 計算每個 video_name 平均有幾個 view_action
video_stats['avg_view_action'] = video_stats['view_action'] / video_stats['review_sn_count']

# 按照 avg_view_action 數量排序
video_stats_sorted = video_stats.sort_values(by='avg_view_action', ascending=False)
display(video_stats_sorted.head())

,video_name,view_action,video_len,review_sn_count,avg_view_action
119,video:5-s-02-S03,169,228.228,2,84.5
174,五年級指代詞-1,118,125.226,2,59.0
69,video:5-n-06-S02,242,585.398,5,48.4
63,video:5-n-02-S03,215,421.559,5,43.0
71,video:5-n-07-S02,203,220.541,5,40.6


### Finding Correlation

In [6]:
# 分析 video_len 和 avg_view_action 之間的相關性
correlation = video_stats['video_len'].corr(video_stats['avg_view_action'])
print(f"Correlation between video length and average view actions: {correlation}")
# 分析 video_len 和 view_action 之間的相關性
correlation_view_action = video_stats['video_len'].corr(video_stats['view_action'])
print(f"Correlation between video length and total view actions: {correlation_view_action}")

Correlation between video length and average view actions: 0.20010843181778315
Correlation between video length and total view actions: 0.22415525465555694


### Conclusion
---
The analysis indicates that there is **no significant correlation** between <u>video_len</u> and <u>view_action_count</u>

It is hypothesized that the number of interactions is related to the difficulty level of the videos. Further analysis is needed to investigate whether `the number of checkpoints` within the videos is correlated with the number of interactions.

### Check Point
---

In [7]:
#print(df_exam['question_sn'].nunique())
# Calculate the number of unique question_sn for each review_sn
max_question_sn_counts = df_exam.groupby('review_sn')['question_sn'].nunique().reset_index()

# Merge video_review_mapping with max_question_sn_counts
video_question_sn_counts = video_review_mapping.merge(max_question_sn_counts, on='review_sn', how='left')

# Calculate the maximum number of unique question_sn for each video_name
video_max_question_sn_counts = video_question_sn_counts.groupby('video_name')['question_sn'].max().reset_index()

# Filter out videos with / without check points
videos_with_checkpoints = video_max_question_sn_counts[video_max_question_sn_counts['question_sn'].notna()]
videos_without_checkpoints = video_max_question_sn_counts[video_max_question_sn_counts['question_sn'].isna()]

# Count the number of videos with / without check points
num_videos_with_checkpoints = videos_with_checkpoints.shape[0]
num_videos_without_checkpoints = videos_without_checkpoints.shape[0]
videos_with_checkpoints = videos_with_checkpoints.merge(video_review_mapping[['video_name', 'video_len']], on='video_name', how='left')
print(f"沒有檢核點的影片數量有: {num_videos_without_checkpoints}", "個")
print(f"有檢核點的影片數量有: {num_videos_with_checkpoints}","個")

沒有檢核點的影片數量有: 157 個
有檢核點的影片數量有: 117 個
